<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Explainble Models with LIME

_Authors: Greg Baker (SYD) and Justin Pounders_

---

### Learning Objectives

- Define some of the political, ethical and economic ramifications of using black-box models in data science.
- Describe the features of explainable vs non-explainable (black-box) models
- Define and discuss the global implications of the EU GDPR
- Use LIME to explain a black-box model

### Lesson Guide
- [Introduction: legal and ethical responsibilities](#intro)
- [Black-box models and explainability](#blackbox)
- [GDPR](#gdpr)
- [Options for Machine Learning on EU Citizen Data](#options)
- [LIME Example](#lime)
- [Resources](#resources)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

<a id='intro'></a>

## Introduction: legal and ethical responsibilities

As data scientists, we often have access to very personal information about 
our customers or users. 

Many countries have very strict laws on how such data must be handled.

### Warning

Laws change -- sometimes quite suddenly -- and General Assembly doesn't
promise that anything in this topic won't be out-of-date by the time you
finish the class.

Also, _I AM NOT A LAWYER_.

## Why do we care?

![](assets/chowdhury.png)

> Taken from Rumman Chowdhury's presentation at the Southern Data Science Conference, 2018.

## OKCupid

![](assets/okcupid.png)

> http://fortune.com/2014/07/28/okcupid-we-experiment-on-users-too/

## Cambridge Analytica

![](assets/cambridge1.png)

---

![](assets/cambridge2.png)

> https://www.nytimes.com/2018/03/19/technology/facebook-cambridge-analytica-explained.html

----

## Racial "profiling" by country

<img src="assets/France.png" style="float: left; margin: 10px; height: 50px">

### France - you must not ask about race

<br>

<blockquote>Race is such a taboo term that a 1978 law specifically banned the collection and computerized storage of race-based data without the express consent of the interviewees or a waiver by a state committee. France therefore collects no census or other data on the race (or ethnicity) of its citizens.  [Race policy in France](https://www.brookings.edu/articles/race-policy-in-france/)
</blockquote>

Even if you don't use race in your model, you simply can't store this in a database or even
have it appear in a pandas dataframe.


----

<img src="assets/malaysia.png" style="float: left; margin: 10px; height: 50px">

### Malaysia - you must ask about race

<br>

<blockquote>
Race is such an important term in Malaysia and the history of discrimination so
strong, that there are great efforts to make sure that companies aren't being discriminatory 
or racist. It is quite
common to collect some quite personal data about every customer, student or user:

<ul>
<li> _What is your race?_
<li> _What is your religion?_
</ul>

These are written on everyone's national identity card and stored in numerous databases.
Changing religion requires completing a formal application and being issued with a new 
identity card.
</blockquote>

If you are dealing with data from Malaysia it will be important to show that any algorithm you 
use doesn't adversely affect people of different religions as a court can easily request data 
on your per-race outcomes. Analysing student education 
outcomes to confirm no racial or religius bias is important, for example.

So you will definitely need to do generate some descriptive statistics on race, for example.

### Side-note:

This makes it very difficult for a French university to open a campus in Malaysia!

## What is the underlying ethical motivation, and how does it affect data scientists?


- Generally, countries set up laws so that the colour of your skin or what you believe shouldn't affect how
you are treated by other human beings.
- But many decisions are now being made by _algorithms_, rather than human beings.

> That's the first rule of algorithms.
> Algorithms are **opinions embedded in code**.
> ...
> (As data scientists), we should not be the arbiters of truth. We should be translators of ethical discussions that happen in larger society.
> --- [Cathy O'Neill TED Talk](https://www.ted.com/talks/cathy_o_neil_the_era_of_blind_faith_in_big_data_must_end/transcript)

- Data scientists have a unique responsibility to make algorithms that **treat everyone fairly**, just as you would
expect a human being to treat everyone fairly.
- In Europe, this is now a **legal responsibility**, which will mostly fall on data scientists: justify why your
model is reasonable and fair.
 

## General Considerations

- Privacy/anonymity
- Data ownership
- Consent
- Explainability

> We are going to focus on explainability for the rest of the lesson.

<a name="blackbox"></a>

----

## What is a black-box model?

A **black-box model** is a machine learning model where it is hard to explain to a non-technical audience how or why it works. It's like looking into a very dark box where you can't see anything.

**An explainable model** is the opposite of a black-box model:

- The workings of the algorithm aren't secret intellectual property

- It is possible for a human being to take the result of training and calculate predictions by hand.

- It only uses a small number of attributes to make its decision -- too many and it's hard for a human being
  to think through their interactions.

- The attributes the algorithm uses to make a prediction seem relevant to the problem at hand


Some explainable examples:

- A linear model $y = a_0 x_0 + a_1 x_y + b$ can be computed by hand even if it required a computer
  to identify the optimal $a_0$ and $a_1$
  
- A decision tree with a depth of 3 (8 nodes) can be traced by hand even if it required a computer
  to find the best variables to split on
  
Some black-box models:
  
- A random forest used the average answer from a thousand different decision trees would be hard to calculate by hand

- The k-nearest-neighbours algorithm is conceptually simple but would be hard to calculate by hand. Also it uses all the attributes of each data point

## Knowledge check

1. When should we _not_ use black-box models?

2. Based on your knowledge of these algorithms, tag them as being "easy to explain" or "a bit of a black-box"

  - Logistic regression

  - Deep learning

  - Random forests

  - Bayesian methods

  - Decision trees

  - Rule lists (sequences of if-then statements)
  
  - k-Nearest-Neighbours

<a id='gdpr'></a>

----

<img src="assets/eu.png" style="float: left; margin: 20px; height: 50px">

# EU GDPR

<br>

The General Data Protection Regulation was agreed to in April 2016. It went into effect on **25th May 2018**.

It puts some very serious constraints on what can and can't be kept in a database, and makes some
common machine learning algorithms difficult to use.

## Article 22

<blockquote>
<p>
The data subject shall have the right not to be subject to a **decision based solely on automated processing**, including profiling, which produces legal effects concerning him or her or similarly significantly affects him or her.
</p>

<p>
The data controller shall implement suitable measures to safeguard the data subject's rights and freedoms and legitimate interests, at least the **right to obtain human intervention** on the part of the controller, to express his or her point of view and to contest the decision. 
</p>
</blockquote>

Since a machine learning classifier is definitely a "decision based solely on automated processing", this means that EU citizens have the right to ask to bypass that wonderful new machine learning algorithm you created and have a human being make the decision instead.

The legislation doesn't strictly define what a "legal effect" is, but at the extremes:

- Choosing the optimal image for a landing page to match someone's preferences is probably OK and probably not subject to Article 22.

- If you have different pricing models for different customers and you use a machine learning algorithm to
 choose the offer that you think will best suit them (or makes your company the most money!) you are probably subject to
 Article 22

## Article 9

<blockquote>
<p>
 Processing of personal data revealing racial or ethnic origin, political opinions, religious or philosophical beliefs, or trade union membership ... data concerning health or data concerning a natural person's sex life or sexual orientation shall be prohibited. 
 </p>
</blockquote>

This applies to _any company, anywhere in the world_ working with data from EU citizens. Your input dataframe ($X$) in 
your model _can't_ have columns for:
 
 - sexual orientation
 - race
 - politics
 - religion
 
You can't even have columns that are strongly correlated with any of these things.


## Article 83

<blockquote>
Infringements of the ... provisions shall ... be subject to administrative fines ... up to 4 % of the total worldwide annual turnover of the preceding financial year
</blockquote>

For reference, this is the maximum potential fine for various companies:

| Company | Annual Revenue | Maximum Fine ||
|----------|---------------|---------------|
| Microsoft (including LinkedIn) | USD100B | USD 4,000,000,000 |
| Alphabet (Google) | USD90B | USD 3,600,000,000 |
| Facebook | USD27B | USD 1,080,000,000 |
| Netflix | USD8.8B | USD 320,000,000 |


## Recital 71

<blockquote>
...processing should be subject to suitable safeguards, which should include ... **an explanation of the decision** reached after such assessment and [the option] to challenge the decision.
</blockquote>

Recitals are not law, they are additional information written out to explain the purposes and thinking of the Articles. It's possible that the EU courts may not interpret the law the same way as the lawmakers did!

But this key phrase: _an explanation of the decision_ has caused a lot of angst in the machine learning community.





## Recital 71 (continued)

<blockquote>
... the controller should use appropriate mathematical or statistical procedures ... [to take] account of the potential risks involved for the interests and rights of the data subject and that prevents, inter alia, discriminatory effects on natural persons on the basis of racial or ethnic origin, political opinion, religion or beliefs, trade union membership, genetic or health status or sexual orientation, or that result in measures having such an effect.
</blockquote>

Not only do you have to show that you didn't deliberately include race, religion or sexual orientation in your model, *you also have to show that your model isn't discriminating against anyone accidentally*. 

Of course, this is a bit challenging
for your French users, because you won't be able to store what race they identify with in the first place.

# Review questions

1. Can you think of any data that you would you not want to store about a person? (Suggestions: think about
things that might be illegal or immoral or embarrassing for them if it were in a database that you controlled.)

2. If you had very complete information about someone (where they live, every address they have
visited in the last year, what sort of car they drive, what they have bought in the last year, every phone number
they have called, everyone they have emailed or messaged), how well could
you guess their religion, politics, sexual orientation or ancestry? How would you do this?

3. If a company or government department made a decision about you that you didn't think was fair, how would you 
like to have your complaint handled?

4. If your company is based outside of the EU, you don't have to worry about the EU GDPR. True or false? 


<a id="options"></a>

----

# Options for Machine Learning on EU Citizen's Data


There are two approaches that are likely to satisfy the GDPR:

- Only use machine learning algorithms that can be understood very easily by non-technical users

- Retro-fit explainability over other algorithms, including black-box algorithms


<a name='lime'></a>

# LIME

You might remember a maths teacher telling you that if you zoom in very close to a typical continuous 
function it will look linear.

No matter how complex a black box model you have created, it is nearly impossible to make something
that isn't generally continuous and still be useful.

So while we can't always explain how a black box model works in general, if you zoom in very close,
you will be able to explain what is happening *locally*.

This is known as LIME: **Locally Interpretable Model-Agnostic Explanations**

> If you haven't already install the `lime` package: `pip install lime`.  
> The package source code (and some good resources like a video introduction and the original paper) are on [GitHub](https://github.com/marcotcr/lime).

## How LIME works

Let's say that we have a hard-to-explain model named $H$, and an observation called $x$ that we would like to explain.

1. Generate a bunch of "close-by" data points by randomly altering the values of features.  Let's call these "perturbed data."
2. For each perturbed data point, predict the label using your hard-to-explain model, $H$.
3. Gather the perturbed data and their labels from steps 1 and 2.  This is a _new training set_ for an explainable model.
4. Train your easy-to-explain model (call it $E$) on your new dataset from 3.
5. Use your easy-to-explain model $E$ to predict the label of $x$.
6. Because $E$ is easy to explain, you can explain the prediction made for $x$.


In the following picture, the bright red cross is the instance being explained, and the other crosses
and dots are other nearby data points. Based on this, it will make a linear rule (a straight line that
divides the two different background colours). It's obviously not right everywhere, but it is correct
close to the big red cross.

<img src="assets/lime.png">

Our explanation is:

- if the red cross had been a little further to the right and down, it would have been classified differently

- if the red cross had been a little further to the left and up, it would have been classified the same

## Who survived the Titanic?

If you wanted to understand why your Titanic model predicted that Henry Tornquist
(3rd class, male, age 25) should have died (he actually survived), LIME would get a prediction from
your model for:

- A 3rd class, male, age 24
- A 3rd class male, age 26
- A 2nd class, male, age 25
- A 3rd class, female, age 25

And so on -- a variety of similar, but not-quite-the-same people. LIME would then discover that

- being male instead of female made a big difference
- being third class instead of 2nd class made a big difference
- being a little older or younger made very little difference

It would explain your model as "mostly responding to Sex and Pclass"

In [ ]:
import pandas as pd
import numpy as np
import sklearn.neighbors
import sklearn.model_selection

knn = sklearn.neighbors.KNeighborsClassifier()
titanic = pd.read_csv('datasets/titanic.csv', index_col=0)
titanic = pd.DataFrame({
    'Age': titanic.Age / titanic.Age.mean(),
    'Pclass': titanic.Pclass,
    'Sex': (titanic.Sex == 'male').astype(int),
    'Survived': titanic.Survived
}).dropna().values

#titanic

(Xtrain, Xtest, Ytrain, Ytest) = sklearn.model_selection.train_test_split(titanic[:,:3], titanic[:,3], random_state=42)
knn.fit(Xtrain, Ytrain)
knn.score(Xtest, Ytest)

In [ ]:
# Get a baseline
values, counts = np.unique(Ytrain, return_counts=True) 
counts/len(Ytrain)

In [ ]:
import lime
import lime.lime_tabular

explainer = lime.lime_tabular.LimeTabularExplainer(Xtrain,
                                                   feature_names = ['Age', 'Pclass', 'Sex'],
                                                   class_names = ['Died', 'Survived'],
                                                   discretize_continuous=True)
explanation = explainer.explain_instance(np.array([25, 3, 1]),
                           knn.predict_proba,
                           num_features=2
                          )
explanation.show_in_notebook()

In [ ]:
explanation.as_list()

The most significant features are:
 - Mr Tornquist's Sex (greater than 0 means male)
 - His ticket class (greater than 2 means third class)
 
We could write this as:

  $survival = -0.47 * maleness - 0.26 * thirdclassness$
  
For passengers who are of a similar age to Mr Tornquist, this provides a reasonable quick rule
of thumb about whether they would be more likely to survive or less like to survive than Mr Tornquist.
Be less male or buy a different ticket if you want to survive.

## Who's on first?

In [ ]:
from sklearn.datasets import fetch_20newsgroups

categories = ['rec.sport.baseball', 
              'rec.sport.hockey']

data_train = fetch_20newsgroups(subset='train', categories=categories)
data_test = fetch_20newsgroups(subset='test', categories=categories)

In [ ]:
# What's baseline?
values, counts = np.unique(data_train.target, return_counts=True)
counts/len(data_train.target)

In [ ]:
# Build a model

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

components = [('vectorizer', TfidfVectorizer(stop_words='english')),
              ('clf', SVC(gamma=1., C=1.0, probability=True))]

pipe = Pipeline(components)
pipe.fit(data_train.data, data_train.target)

In [ ]:
pipe.score(data_train.data, data_train.target), pipe.score(data_test.data, data_test.target)

In [ ]:
from lime.lime_text import LimeTextExplainer

explainer = LimeTextExplainer(class_names=categories)

In [ ]:
idx = 0
idx = 2

exp = explainer.explain_instance(data_test.data[idx], pipe.predict_proba, num_features=6)
print('Document id: %d' % idx)
print('Probability(hockey) =', pipe.predict_proba([data_test.data[idx]])[0,1])
print('True class: %s' % categories[data_test.target[idx]])

In [ ]:
exp.as_list()

In [ ]:
fig = exp.as_pyplot_figure()

In [ ]:
exp.show_in_notebook(text=True)

<a id='resources'></a>

## Additional resources

---


- [Is there are a right to explanation for Machine Learning in the GDPR?](https://iapp.org/news/a/is-there-a-right-to-explanation-for-machine-learning-in-the-gdpr)
- [Why a Right to Explanation of Automated Decision-Making Does Not Exist in the General Data Protection Regulation](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2903469)
- [Towards Interpretable Reliable Models](https://blog.kjamistan.com/towards-interpretable-reliable-models/)
- [GDPR and you](https://blog.kjamistan.com/gdpr-you-my-talk-at-cloudera-sessions-munchen/)
- [Hold Your Machine Learning and AI Models Accountable](https://medium.com/pachyderm-data/hold-your-machine-learning-and-ai-models-accountable-de887177174c)
- [How GDPR Affects Data Science](https://kdnuggets.com/2017/07/gdpr-affects-data-science.html)
- [Scaleable Bayesian rule lists](https://arxiv.org/pdf/1602.08610v2.pdf)

- [Why Should I Trust You? Explaining the Predictions of Any Classfier ](https://www.youtube.com/watch?v=hUnRCxnydCc)
- [Explaining Complex Machine Learning Models with LIME](https://datascienceplus.com/explaining-complex-machine-learning-models-with-lime/)

---

Other methods that can be used to support model explainability:

- [Shapley values](https://en.wikipedia.org/wiki/Shapley_value) emerged from game theory.  They have recently been applied to machine learning to explain which features were important in arriving at a prediction.  There is also a Python implementation: [SHAP (SHapley Additive exPlanations)](https://github.com/slundberg/shap).
- [Path Specific Counterfactual Fairness](https://arxiv.org/abs/1802.08139) is a recent paper that looks at _direct_ and _indirect_ effects to determine whether certain features are actually influencing a prediction.  For example, this method can be used to demonstrate that gender bias was apparently _not_ a factor in Berkley admissions because the "counterfactual" case (in which the gender features were switched) did not affect the prediction.
- [Generating Visual Explanations](https://arxiv.org/abs/1603.08507) uses a combination of images and text to automatically generate _useful_ textual descriptions of why predictions were made in image classification.

![](assets/viz_exp.png)